In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [5]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator
from intelligence_tools import AnalyzeVariables

In [6]:
df = pd.read_csv('../datasets/drug_consumption.csv')

In [7]:
av = AnalyzeVariables(df)

In [9]:
av.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1885 entries, 0 to 1884
Data columns (total 32 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         1885 non-null   int64  
 1   Age        1885 non-null   float64
 2   Gender     1885 non-null   float64
 3   Education  1885 non-null   float64
 4   Country    1885 non-null   float64
 5   Ethnicity  1885 non-null   float64
 6   Nscore     1885 non-null   float64
 7   Escore     1885 non-null   float64
 8   Oscore     1885 non-null   float64
 9   Ascore     1885 non-null   float64
 10  Cscore     1885 non-null   float64
 11  Impulsive  1885 non-null   float64
 12  SS         1885 non-null   float64
 13  Alcohol    1885 non-null   object 
 14  Amphet     1885 non-null   object 
 15  Amyl       1885 non-null   object 
 16  Benzos     1885 non-null   object 
 17  Caff       1885 non-null   object 
 18  Cannabis   1885 non-null   object 
 19  Choc       1885 non-null   object 
 20  Coke    

In [10]:
target_columns = ['Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms','Nicotine', 'Semer', 'VSA']

In [11]:
for col in target_columns:
    df[col] = pd.to_numeric(df[col].str.replace('CL', '', regex=False), errors='coerce')

In [15]:
av.missing_percentage()

ID           0.0
Age          0.0
Gender       0.0
Education    0.0
Country      0.0
Ethnicity    0.0
Nscore       0.0
Escore       0.0
Oscore       0.0
Ascore       0.0
Cscore       0.0
Impulsive    0.0
SS           0.0
Alcohol      0.0
Amphet       0.0
Amyl         0.0
Benzos       0.0
Caff         0.0
Cannabis     0.0
Choc         0.0
Coke         0.0
Crack        0.0
Ecstasy      0.0
Heroin       0.0
Ketamine     0.0
Legalh       0.0
LSD          0.0
Meth         0.0
Mushrooms    0.0
Nicotine     0.0
Semer        0.0
VSA          0.0
dtype: float64

In [16]:
bdf = BaseDatasetTransform(df,target=['Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms',
       'Nicotine', 'Semer', 'VSA'])

In [17]:
X_train, X_test, y_train, y_test,categorical_features = bdf.fit_transform()


Обнаружены колонки с именем "id": ['ID']
Колонка "ID" удалена.
Series([], dtype: int64)

В колонке(ах) ['Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms', 'Nicotine', 'Semer', 'VSA'] нет пропущенных значений
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['Alcohol', 'Amphet', 'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc',
       'Coke', 'Crack'],
      dtype='object')

Числовые колонки:
Index(['Age', 'Gender', 'Education', 'Country', 'Ethnicity', 'Nscore',
       'Escore', 'Oscore', 'Ascore', 'Cscore', 'Impulsive', 'SS'],
      dtype='object')


In [18]:
from sklearn.multioutput import MultiOutputClassifier


lgbm_model = MultiOutputClassifier(lgb.LGBMClassifier(verbose=-1))

columns_to_delete = []  

unique_classes = len(set(y_train))
print(unique_classes)

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    #objective='binary',  
    #metric='binary_error' 
)

10


In [19]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics,base_multitarget=True)

evaluator.fit(X_train, y_train)

ModelEvaluator(base_multitarget=True,
               metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['Alcohol',
                                                                  'Amphet',
                                                                  'Amyl',
                                                                  'Benzos',
                                                                  'Caff',
                                                                  'Cannabis',
                                                                  'Choc',
                                                                  'Coke',
                                                                  'Crack'],
                                            columns_to_delete=[],
                                            model=MultiOutputClassifier(estimator=LGBMClassifier(verbose=-1))))

In [20]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,accuracy,precision,recall,f1_macro,Targets
0,0.60424,0.325147,0.286855,0.296402,Ecstasy
1,0.839223,0.212204,0.174114,0.180049,Heroin
2,0.786219,0.28183,0.195527,0.208247,Ketamine
3,0.581272,0.215556,0.213898,0.205683,Legalh
4,0.639576,0.329797,0.311369,0.315952,LSD
5,0.742049,0.322678,0.199916,0.211242,Meth
6,0.584806,0.300544,0.30167,0.296532,Mushrooms
7,0.429329,0.283796,0.262817,0.258403,Nicotine
8,0.9947,0.331567,0.333333,0.332448,Semer
9,0.750883,0.194413,0.171523,0.172594,VSA
